## Lab assignment 02

### Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the EN-RU translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* Transformer/CNN/<whatever you select> encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language models (for decoder and encoder)

* or just fine-tunning BART/ELECTRA/... ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [1]:
# You might need to install the libraries below. Do it in the desired environment
# if you are working locally.

! pip  install subword-nmt
! pip install nltk
! pip install torchtext==0.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# Thanks to YSDA NLP course team for the data
# (who thanks tilda and deephack teams for the data in their turn)

import os
path_do_data = '../../datasets/Machine_translation_EN_RU/data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc
    path_do_data = './data.txt'

Dataset not found locally. Downloading from github.
--2023-05-21 19:36:31--  https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12905334 (12M) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  12.31M  --.-KB/s    in 0.06s   

2023-05-21 19:36:31 (213 MB/s) - ‘data.txt’ saved [12905334/12905334]



Baseline solution BLEU score is quite low. Try to achieve at least __21__ BLEU on the test set. 
The checkpoints are:

* __21__ - minimal score to submit the homework, 30% of points

* __25__ - good score, 70% of points

* __27__ - excellent score, 100% of points

### Warning! The code below is deeeeeeeply deprecated and is is provided only as simple guide.
We suggest you to stick to most recent pipelines here, e.g. by Huggingface: 
* Example notebook: [link](https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb)
* Converting your own dataset to specific format: [link](https://discuss.huggingface.co/t/correct-way-to-create-a-dataset-from-a-csv-file/15686/15)

# Seq2Seq

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu

In [3]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ru-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/1516162 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2998 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ru-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-3-ffa90e77666a>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [4]:
import datasets
import transformers
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
import pandas as pd

def text2df(file_name):
  translations = []
  file = open(file_name, "r")
  while True:
      line = file.readline()
      if not line:
          break
      else:
        dict = {}
        splited_line = line.split("\t")
        dict['en'] = splited_line[0]
        dict['ru'] = splited_line[1]
        translations.append(dict)
  file.close
  df = pd.DataFrame({'translation': translations})
  return df

In [37]:
import pandas as pd

def text2csv(file_name):
  translations = []
  file = open(file_name, "r")
  dictdf = {'en':[], 'ru':[]}
  while True:
      line = file.readline()
      if not line:
          break
      else:
        splited_line = line.split("\t")
        dictdf['en'].append(splited_line[0])
        dictdf['ru'].append(splited_line[1])
  file.close
  df = pd.DataFrame(dictdf, index=list(range(len(dictdf['en']))))
  return df

In [38]:
df = text2csv('data.txt')

In [40]:
df.to_csv('data.csv')

In [6]:
df = text2df('data.txt')

In [7]:
df.head()

,translation
0,"{'en': 'Cordelia Hotel is situated in Tbilisi,..."
1,{'en': 'At Tupirmarka Lodge you will find a 24...
2,"{'en': 'Featuring free WiFi in all areas, Naig..."
3,{'en': 'Each has a TV and a private bathroom w...
4,{'en': 'Your room comes with air conditioning ...


In [9]:
from transformers import AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
tokenizer("Hello, this one sentence!")

{'input_ids': [160, 5270, 2, 66, 159, 4087, 56, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ru"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.05)
train, valid = train_test_split(df, train_size=0.8*0.95)

In [15]:
from datasets import Dataset
train = Dataset.from_pandas(train, preserve_index=False)
test = Dataset.from_pandas(test, preserve_index=False)
valid = Dataset.from_pandas(valid, preserve_index=False)

In [16]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)
tokenized_train

Map:   0%|          | 0/38000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 38000
})

In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [18]:
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [19]:
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [20]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # remove_unused_columns=False
    # fp16=True,
    # push_to_hub=True,
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [23]:
trainer = Seq2SeqTrainer(
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.247200,0.229774,33.872000,27.772800


TrainOutput(global_step=2375, training_loss=0.2913795744243421, metrics={'train_runtime': 1836.6166, 'train_samples_per_second': 20.69, 'train_steps_per_second': 1.293, 'total_flos': 1079311227420672.0, 'train_loss': 0.2913795744243421, 'epoch': 1.0})

### Test

In [27]:
test_trainer = Seq2SeqTrainer(model=model,
                              args=args,
                              data_collator=data_collator)

raw_pred, _, _ = test_trainer.predict(tokenized_test)

### Calculate BLEU

In [29]:
labels = tokenized_test['labels']
if isinstance(raw_pred, tuple):
        y_pred = raw_pred[0]
decoded_preds = tokenizer.batch_decode(raw_pred, skip_special_tokens=True)
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Some simple post-processing
decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

result = metric.compute(predictions=decoded_preds, references=decoded_labels)
result = {"bleu": result["score"]}

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [30]:
result

{'bleu': 35.057281983590045}

# facebook/wmt19-en-ru

In [13]:
from datasets import load_dataset, load_metric
metric = load_metric("sacrebleu")

In [ ]:
! pip install sacremoses

In [10]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-en-ru"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

input = "Machine learning is great, isn't it?"
input_ids = tokenizer.encode(input, return_tensors="pt")
outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
result = metric.compute(predictions=decoded_preds, references=decoded_labels)
print(decoded) # Машинное обучение - это здорово, не так ли?

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Машинное обучение - это здорово, не так ли?


In [42]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data.csv')

train, test = train_test_split(df, test_size=0.05)

In [45]:
test

,Unnamed: 0,en,ru
13987,13987,The hotel bar provides a relaxing environment ...,"В баре отеля царит расслабляющая атмосфера, ко..."
43681,43681,Rio de Janeiro/Galeao International Airport is...,Расстояние до Международного аэропорта Рио-де-...
34974,34974,The unit is fitted with a kitchenette.,В апартаментах есть мини-кухня и собственная в...
43236,43236,The Brookfield Zoo along with Chicago city cen...,До зоопарка Брукфилд и центра Чикаго тоже всег...
966,966,The à la carte restaurant on site serves tasty...,В ресторане отеля с обслуживанием по меню гото...
...,...,...,...
36587,36587,Each is provided with a welcome kit including ...,Гостям предоставляется приветственный набор с ...
25160,25160,Hotel Gerard D'alsace also features free Wi-Fi...,На территории отеля Gerard D'alsace предоставл...
8217,8217,"All rooms offer cable TV, a minibar, bathrobes...","Во всех номерах есть кабельное телевидение, ми..."
43909,43909,Other facilities offered at the property inclu...,Для удобства гостей оборудована камера хранени...


In [52]:
import numpy as np

In [53]:
np.array(test.iloc[:,2])

array(['В баре отеля царит расслабляющая атмосфера, которая позволяет расслабиться с любимым напитком после дня знакомства с центром Лондона.\n',
       'Расстояние до Международного аэропорта Рио-де-Жанейро — Галеан составляет 20 км.\n',
       'В апартаментах есть мини-кухня и собственная ванная комната с ванной и биде.\n',
       ...,
       'Во всех номерах есть кабельное телевидение, мини-бар, фен и рабочий стол. Гостям предоставляются купальные халаты. В большинстве номеров имеется балкон с впечатляющим видом на горы.\n',
       'Для удобства гостей оборудована камера хранения багажа и бесплатная парковка.\n',
       'Гости могут прокатиться на велосипеде или на лодке по близлежащей реке Мозель.\n'],
      dtype=object)

In [57]:
import numpy as np
from tqdm import tqdm

preds = []
# for i in tqdm(range(len(test))):
for i in tqdm(range(4)):
  input = test.iloc[i,1]
  input_ids = tokenizer.encode(input, return_tensors="pt")
  outputs = model.generate(input_ids)
  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  preds.append(decoded)

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 4/4 [00:14<00:00,  3.62s/it]


In [67]:
preds = np.array(preds, dtype=object)

In [69]:
preds

array(['Бар отеля предоставляет расслабляющую атмосферу, чтобы расслабиться с напитком после дня, проведенного в центре Лондона.',
       'Международный аэропорт Рио-де-Жанейро / Галеао расположен в 20 км от отеля.',
       'Аппарат оснащен котенком.',
       'Зоопарк Брукфилд, а также центр Чикаго также находятся в нескольких минутах езды от отеля.'],
      dtype=object)

In [66]:
np.array(test.iloc[:4,2])

array(['В баре отеля царит расслабляющая атмосфера, которая позволяет расслабиться с любимым напитком после дня знакомства с центром Лондона.\n',
       'Расстояние до Международного аэропорта Рио-де-Жанейро — Галеан составляет 20 км.\n',
       'В апартаментах есть мини-кухня и собственная ванная комната с ванной и биде.\n',
       'До зоопарка Брукфилд и центра Чикаго тоже всего пара минут езды.\n'],
      dtype=object)

In [70]:
preds = np.array(preds)
result = metric.compute(predictions=preds, references=np.array(test.iloc[:4,2]))
result = {"bleu": result["score"]}
prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
result["gen_len"] = np.mean(prediction_lens)
result = {k: round(v, 4) for k, v in result.items()}
result

ValueError: ignored

## Вывод

Дообученная на датасете модель видимо работает лучше (см. "Аппарат оснащен котенком")